<a href="https://colab.research.google.com/github/drusho/100daysofcode/blob/main/Exploring_Car_Sales_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Ebay Car Sales Data

Data comes from a modified csv containing informaiton on used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

<br>

### __Project Goal:__
- Clean raw data and analyze the used car listings.
<br>

### __Results__


In [104]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/drusho/100daysofcode/main/csv_files/Exploring_Ebay_Car_Sales_Data.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9392k  100 9392k    0     0  13.9M      0 --:--:-- --:--:-- --:--:-- 13.9M


In [105]:
# Read in the data
import pandas as pd
import numpy as np


df = pd.read_csv('Exploring_Ebay_Car_Sales_Data.csv', encoding='Latin-1')
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


#### __Description of Column Headings__
|Column  |Description |
|:-- |:-- |
|*dateCrawled* | When this ad was first crawled. All field-values are taken from this date. |
|*name*|Name of the car |
|*seller*|Whether the seller is private or a dealer.|
|*offerType*|The type of listing|
|*price*|The price on the ad to sell the car.|
|*abtest*|Whether the listing is included in an A/B test.
|*vehicleType*|The vehicle Type.|
|*yearOfRegistration*|The year in which the car was first registered.|
|*gearbox*|The transmission type.|
|*powerPS*|The power of the car in PS.|
|*model*|The car model name.|
|*kilometer*|How many kilometers the car has driven.|
|*monthOfRegistration*|The month in which the car was first registered.|
|*fuelType*|What type of fuel the car uses.|
|*brand*|The brand of the car.|
|*notRepairedDamage|If the car has a damage which is not yet repaired.|
|*dateCreated*|The date on which the eBay listing was created.|
|*nrOfPictures*|The number of pictures in the ad.|
|*postalCode*|The postal code for the location of the vehicle.|
|*lastSeenOnline*|When the crawler saw this ad last online.|

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

### __* Initial Observations of the Raw Data__
- The dataset contains 20 columns.  
- The majority of the data is stored as strings. 
- A few columns contain null values (ex. *vehicleType, gearbox, model*)
- None of the columns have more than ~20% null values (based off 49,999 entries compared to non-null counts)
- There are some columns that contain dates stored as strings.


### **Clean Columns**
- Camelcase to snakecase.
- Renaming some columns so that the headings reflect the data they contain

_examples:_
- _yearOfRegistration_ → _registration_year_
-  _odometer_ → _odometer_km_

In [107]:
# convert the column names from camelcase to snakecase

column_names = []
for column in df.columns:
  column_names.append(column.lower())
df.columns = column_names

df.columns

Index(['datecrawled', 'name', 'seller', 'offertype', 'price', 'abtest',
       'vehicletype', 'yearofregistration', 'gearbox', 'powerps', 'model',
       'odometer', 'monthofregistration', 'fueltype', 'brand',
       'notrepaireddamage', 'datecreated', 'nrofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [108]:
# rename some column names to increase fluency

df.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']

df.columns


Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen'],
      dtype='object')

In [109]:
df.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-14 20:50:02,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


### __* Observations__


Columns that have mostly one value (potiental candidates to be dropped):
- **num_photos** (0 for every row)
- _offer_type_
- _seller_

Columns with numeric data stored as text that needs to be cleaned
- _price_
- _odometer_

In [110]:
# drop columns with 1 or fewer values
 
df.drop(columns=['num_photos', 'offer_type', 'seller'], axis=1,inplace=True)
df.head()

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


In [111]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_crawled        50000 non-null  object
 1   name                50000 non-null  object
 2   price               50000 non-null  object
 3   ab_test             50000 non-null  object
 4   vehicle_type        44905 non-null  object
 5   registration_year   50000 non-null  int64 
 6   gearbox             47320 non-null  object
 7   power_ps            50000 non-null  int64 
 8   model               47242 non-null  object
 9   odometer            50000 non-null  object
 10  registration_month  50000 non-null  int64 
 11  fuel_type           45518 non-null  object
 12  brand               50000 non-null  object
 13  unrepaired_damage   40171 non-null  object
 14  ad_created          50000 non-null  object
 15  postal_code         50000 non-null  int64 
 16  last_seen           50

In [112]:
df['odometer'].value_counts()

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64

In [113]:
df['price'].value_counts()

$0         1421
$500        781
$1,500      734
$2,500      643
$1,200      639
           ... 
$32,490       1
$10,488       1
$4,333        1
$29,777       1
$83,000       1
Name: price, Length: 2357, dtype: int64

In [114]:
# 'price' column has numeric values with extra characters being stored as text.
# remove '$' , ',' and convert to int type
df['price'] = df['price'].str.replace('$','').str.replace(",","").astype(int)
df['price'].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [115]:
# 'odometer' column has numeric values with extra characters being stored as text.
# remove 'km' , ',' and convert to float type
df['odometer'] = df['odometer'].str.replace("km","").str.replace(",","").astype(int)
df['odometer'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64

In [116]:
# rename odometer to reflect data in column (kilometers)
df.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
df.columns

Index(['date_crawled', 'name', 'price', 'ab_test', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')

### **Analyze Odometer and Price**

In [117]:
print(df["price"].unique().shape)
print(df["price"].describe())
df['price'].value_counts(ascending=False).head(10)

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
Name: price, dtype: int64

**$0** accounts for 1421 cars or just 2.8 % of all cars in the _'prices'_ column.

Many of the number values for the 'price' column seem to be rounded, however given there are 2357 unique values in the column, this may be a common trend for this website (Ebay.com).

The maximum price is one hundred million dollars.

In [123]:
print(df['price'].value_counts().sort_index(ascending=False).head(20))
df['price'].value_counts().sort_index(ascending=True).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64


0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

### **'Price' Column Observations**

- 156 cars are listed for $1.00

- $350,000 seems to be point at which prices stop gradually increasing.  

- The next price listed after 350k is $999,990 or just under 1 million.  

##### **Conclusion:**
All 'price' data outside of the 1.00 - 350,000 dollar range will be excluded from further analysis.  This includes:

- 14 cars listed near or above 1 million.
- 1,421 cars listed at 0.00 dollars




In [130]:
# update df to show only prices between $1 - $351,000 
df = df[df['price'].between(1,351000)]
df['price'].value_counts()

500      781
1500     734
2500     643
1200     639
1000     639
        ... 
14998      1
15470      1
9325       1
23650      1
566        1
Name: price, Length: 2346, dtype: int64

,price,registration_year,power_ps,odometer_km,registration_month,postal_code
count,48565.000000,48565.000000,48565.000000,48565.000000,48565.000000,48565.000000
mean,5888.935591,2004.755421,117.197158,125770.101925,5.782251,50975.745207
std,9059.854754,88.643887,200.649618,39788.636804,3.685595,25746.968398
min,1.000000,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1200.000000,1999.000000,71.000000,125000.000000,3.000000,30657.000000
50%,3000.000000,2004.000000,107.000000,150000.000000,6.000000,49716.000000
75%,7490.000000,2008.000000,150.000000,150000.000000,9.000000,71665.000000
max,350000.000000,9999.000000,17700.000000,150000.000000,12.000000,99998.000000


In [103]:
df['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

Odometer column numbers are all rounded. This data was likely recorded using predetermined options for this field. 

**150,000** (the max) is very common, making up both  50% and 75% fields.  Overall there are more high mileage than low mileage values.